In [1]:
import numpy as np
import pandas as pd
import mysql.connector
import gc, os
import jsons
import seaborn as sns
import networkx as nx
import datetime
import uuid
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import joblib
%matplotlib inline

In [14]:
metrics = pd.read_excel("enron_metrics.xlsx")


In [15]:
class Entity:
    def __init__(self, entity_id):
        self.type = "entity"
        self.entity_id = entity_id
        self.dp = {"entity_id":self.entity_id, "type":self.type}
        
    def to_dict(self):
        return (self.dp)
        
        
class Person(Entity):
    def __init__(self, entity_id, full_name):
        super().__init__(entity_id)
        self.dp["object_name"] = self.object_name = "person"
        self.dp["full_name"] = self.full_name = full_name
   
   
    
class Position(Entity):
    def __init__(self, entity_id, position):
        super().__init__(entity_id)
       
        self.dp["object_name"] = self.object_name = "position"
        self.dp["position"] = self.position = position
    

class Attribute(Entity):
    def __init__(self, entity_id, hub_score, is_good_hub, auth_score, is_good_auth, betweenness_score, is_middle_person):
        super().__init__(entity_id)

        self.dp["object_name"] = self.object_name = "attribute"
        self.dp["hub_score"] = self.hub_score = hub_score
        self.dp["is_good_hub"] = self.is_good_hub = is_good_hub
        self.dp["auth_score"] = self.auth_score = auth_score 
        self.dp["is_good_auth"] = self.is_good_auth = is_good_auth
        self.dp["betweenness_score"] = self.betweenness_score = betweenness_score
        self.dp["is_middle_person"] = self.is_middle_person = is_middle_person
                
        
class Email(Entity):
    def __init__(self, entity_id, subject):
        super().__init__(entity_id)
        self.dp["object_name"] = self.object_name = "email"
        self.dp["subject"] = self.subject = subject

    

class Topic(Entity):
    def __init__(self, entity_id, topic):
        super().__init__(entity_id)
        self.dp["object_name"] = self.object_name = "topic"
        self.dp["topic"] = self.topic = topic
          
        

class Relationship:
    def __init__(self, relationship_id):
        self.type = "relationship"
        self.relationship_id = relationship_id
        self.dp = {"relationship_id":self.relationship_id, "type":self.type}
    def to_dict(self):
        return (self.dp)
        
    
class Sends(Relationship):
    def __init__(self, relationship_id, datetime):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "sends"
        self.dp["datetime"] = self.datetime = datetime
        self.dp["year"] = self.year = datetime.year
        self.dp["month"] = self.month = datetime.month
        self.dp["day"] = self.day = datetime.day
        
        
class Receives(Relationship):
    def __init__(self, relationship_id, datetime):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "receives"
        self.dp["datetime"] = self.datetime = datetime
        self.dp["year"] = self.year = datetime.year
        self.dp["month"] = self.month = datetime.month
        self.dp["day"] = self.day = datetime.day
        
class CCed(Relationship):
    def __init__(self, relationship_id, datetime):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "CCed"
        self.dp["datetime"] = self.datetime = datetime
        self.dp["year"] = self.year = datetime.year
        self.dp["month"] = self.month = datetime.month
        self.dp["day"] = self.day = datetime.day
        
class BCCed(Relationship):
    def __init__(self, relationship_id, datetime):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "BCCed"
        self.dp["datetime"] = self.datetime = datetime
        self.dp["year"] = self.year = datetime.year
        self.dp["month"] = self.month = datetime.month
        self.dp["day"] = self.day = datetime.day

class Is(Relationship):
    def __init__(self, relationship_id):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "is"

class Has(Relationship):
    def __init__(self, relationship_id):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "has"
        
class Contains(Relationship):
    def __init__(self, relationship_id, weight):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "contains"
        self.dp["weight"] = self.object_name = weight

class Interacts_with(Relationship):
    def __init__(self, relationship_id, weight):
        super().__init__(relationship_id)
        self.dp["object_name"] = self.object_name = "interacts_with"
        self.dp["weight"] = self.weight = weight

        
        

In [16]:
cnx = mysql.connector.connect(user='root', password='shawn672000',
                              host='localhost',
                              database='enron')
g = cnx.cursor()

q =  "SELECT * FROM message"
msgs = pd.read_sql(con=cnx, sql=q)

msgs["mid"] = msgs.mid.apply(lambda x: str(x)+"email")
msgs.drop("message_id", axis=1, inplace=True)

q = "SELECT * FROM recipientinfo"
recp = pd.read_sql(con=cnx, sql=q)
recp["rid"] = recp["rid"].apply(lambda x: str(x)+"empl")
recp["mid"] = recp["mid"].apply(lambda x: str(x)+"email")

q = """SELECT DISTINCT * FROM
(SELECT eid,  concat(firstName, ' ', lastName) AS employee_name, Email_id as email_id, status AS position FROM enron.employeelist UNION
SELECT eid,  concat(firstName, ' ', lastName) AS employee_name, Email2 as email_id, status AS position FROM enron.employeelist UNION
SELECT eid, concat(firstName, ' ', lastName) AS employee_name, Email3 as email_id, status AS position FROM enron.employeelist UNION
SELECT eid, concat(firstName, ' ', lastName) AS employee_name, Email4 as email_id, status AS position FROM enron.employeelist) s"""

emp = pd.read_sql(con=cnx, sql=q)
emp["eid"] = emp["eid"].apply(lambda x:str(x)+"empl")

In [17]:
msgs = msgs.merge(emp, how="inner", right_on="email_id", left_on="sender")
recp = recp.merge(emp, how = "inner", left_on="rvalue", right_on="email_id")
msgs = msgs.merge(recp, how="inner", left_on="mid", right_on="mid", suffixes=["_msg", "_rec"])
msgs = msgs[msgs.eid_msg != msgs.eid_rec].reset_index(drop=True)



In [18]:
emp = emp[["eid", "employee_name", "position"]].drop_duplicates().reset_index(drop=True)

In [19]:

joblib.dump(value=msgs, filename="msgs.joblib")

['msgs.joblib']

In [2]:
msgs = joblib.load("msgs.joblib")


In [5]:
msgs.to_csv("msgs.csv", sep="\t")

In [20]:
G = nx.MultiDiGraph()
emp.position = emp.position.fillna("N/A")
for p in emp.position:
    pos = Position(entity_id=p, position=p)
    G.add_node(pos.entity_id, attr=pos.to_dict())

for i in range(len(emp)):
    p = Person(emp.iloc[i]["eid"], emp.iloc[i]["employee_name"])
    G.add_node(p.entity_id, attr=p.to_dict())
    ep = Is(relationship_id=uuid.uuid1())
    G.add_edge(u_for_edge=p.entity_id, v_for_edge=emp.iloc[i]["position"], attr=ep.to_dict())

for i in range(len(msgs[["mid", "subject", "body"]])):
    msg = msgs[["mid", "subject"]].iloc[i]
    em = Email(entity_id=msg["mid"], subject=msg["subject"])
    G.add_node(em.entity_id, attr=em.to_dict())
    

msgs_tmp = msgs[["mid", "date", "eid_msg", "rtype", "eid_rec"]]


for i in range(len(msgs_tmp)):
    obj = msgs_tmp.iloc[i]
    if obj["rtype"] == "TO":
        sd = Sends(datetime=obj.date, relationship_id=str(uuid.uuid1()))
        G.add_edge(u_for_edge=obj["eid_msg"], v_for_edge=obj["mid"], attr=sd.to_dict())
        rd = Receives(datetime=obj.date, relationship_id=str(uuid.uuid1()))
        G.add_edge(u_for_edge=obj["eid_rec"], v_for_edge=obj["mid"], attr=rd.to_dict())
    elif obj["rtype"] == "CC":
        rd = CCed(datetime=obj.date, relationship_id=str(uuid.uuid1()))
        G.add_edge(u_for_edge=obj["mid"], v_for_edge=obj["eid_rec"], attr=rd.to_dict())
    elif obj["rtype"] == "BCC":
        rd = BCCed(datetime=obj.date, relationship_id=str(uuid.uuid1()))
        G.add_edge(u_for_edge=obj["mid"], v_for_edge=obj["eid_rec"], attr=rd.to_dict())


In [21]:
edges_id = msgs[["eid_msg", "eid_rec"]].reset_index(drop=True)
edges_id["weight"] = 0
edges_id = edges_id.groupby(by=["eid_msg", "eid_rec"]).count().reset_index()

for i in range(len(edges_id)):
    obj = edges_id.iloc[i]
    iw = Interacts_with(relationship_id=str(uuid.uuid1()), weight=obj["weight"])
    G.add_edge(u_for_edge=obj["eid_msg"], v_for_edge=obj["eid_rec"], attr=iw.to_dict())

emp = emp[["eid", "employee_name"]].drop_duplicates().reset_index(drop=True)
metrics = metrics.merge(emp, left_index=True, right_on="employee_name", how="inner")

metrics_grp = metrics[['hub_auth_clus', "hub", "auth"]].groupby(by=['hub_auth_clus']).mean()

metrics_grp["argmax"] = metrics_grp[["hub", "auth"]].idxmax(axis=1)

high_auth_clus = metrics_grp[metrics_grp["argmax"] == "auth"].sort_values(by="auth", ascending=False).head(1).index[0]
high_hub_clus = metrics_grp[metrics_grp["argmax"] == "hub"].sort_values(by="hub", ascending=False).head(1).index[0]

metrics_grp["is_good_hub"] = "no"
metrics_grp.at[high_hub_clus, "is_good_hub"] = "yes"
metrics_grp["is_good_auth"] = "no"
metrics_grp.at[high_auth_clus, "is_good_auth"] = "yes"

metrics_grp.drop(["hub", "auth", "argmax"], axis=1, inplace=True)
metrics_grp_bet = metrics[['close_and_between_clus', "c_betweenness"]].groupby(by=['close_and_between_clus']).mean()
is_middle = metrics_grp_bet.sort_values(by=["c_betweenness"], ascending=False).head(1).index.values
metrics_grp_bet["is_middle_person"] = "no"
metrics_grp_bet.at[is_middle, "is_middle_person"] = "yes"
metrics_grp_bet.drop("c_betweenness", axis=1, inplace=True)


metrics = metrics[['c_betweenness', 'hub', 'auth', 'close_and_between_clus', 'hub_auth_clus', 'eid']]
metrics[["is_good_hub", "is_good_auth"]] = metrics.hub_auth_clus.apply(lambda x: metrics_grp.loc[x])
metrics["is_middle_person"] = metrics.close_and_between_clus.apply(lambda x: metrics_grp_bet.loc[x])
metrics["attr_id"] = metrics.eid.str.replace("empl", "attr")

metrics.drop(["close_and_between_clus", "hub_auth_clus"], axis=1, inplace=True)

for i in range(len(metrics)):
    obj = metrics.iloc[i]
    att = Attribute(auth_score=obj["auth"], entity_id=obj["attr_id"], is_good_auth=obj["is_good_auth"], 
                    is_middle_person=obj["is_middle_person"], is_good_hub=obj["is_good_hub"], 
                    betweenness_score=obj["c_betweenness"], hub_score=obj["hub"])
    
    G.add_node(att.entity_id, attr=att.to_dict())

for i in range(len(metrics)):
    obj = metrics.iloc[i]
    h = Has(relationship_id=str(uuid.uuid1()))
    G.add_edge(u_for_edge=obj["eid"], v_for_edge=obj["attr_id"], attr=h.to_dict())
    
    

In [22]:
nx.write_gpickle(G=G, path="enron_KG.gpickle")

# Add topics

In [23]:
G = nx.read_gpickle(path="enron_KG.gpickle")
mid_topics = np.load('mid_topics.npz')

mids = mid_topics["mid"]
topics = mid_topics["topics"]
email_topics = mid_topics["email_topics"]
feature_names = mid_topics["feature_names"]

In [24]:
topics = np.apply_along_axis(arr = topics[:,:,0], axis=1, func1d=lambda x:",".join(x))

In [25]:
for i in range(topics.shape[0]):
    tp = Topic(entity_id=i, topic=topics[i])
    G.add_node(tp.entity_id, attr=tp.to_dict())
    

In [26]:
email_t = np.argmax(email_topics, axis=1)
weight = np.max(email_topics, axis=1)
for mid, tp, wt in zip(mids, email_t, weight):
          
    con = Contains(relationship_id=str(uuid.uuid1()), weight=wt)
    G.add_edge(mid, tp, attr=con.to_dict())
 

In [27]:
nx.write_gpickle(G=G, path="enron_KG_full.gpickle")